<a href="https://colab.research.google.com/github/forouzanfallah/Convert_PSCAD_Out_to_CSV/blob/main/Convert_Out_to_NPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


###Directories

In [ ]:
import zipfile
import os

# Replace this with the path to your ZIP file
zip_file_path = '/content/drive/MyDrive/CIGRE-Forouzan/data.zip'

# Replace this with the path where you want to extract the files and store CSV files
data_directory = '/content/CIGRE'

os.makedirs(data_directory, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_directory)

print(f"Files extracted to {data_directory}")


Files extracted to /content/CIGRE


###OUT to NPY

In [ ]:
import pandas as pd
import re
import os

def extract_desc(line):
    match = re.search(r'Desc="([^"]+)"', line)
    if match:
        return match.group(1)
    return None
for folder in os.listdir(data_directory):
    folder_path = os.path.join(data_directory, folder)

    if os.path.isdir(folder_path):
        # List .out files in the directory
        file_names = [f for f in os.listdir(folder_path) if f.endswith('.out')]
        file_names.sort(key=lambda x: int(re.search(r'_(\d+)', x).group(1)))  # Sorting to maintain order
        print(file_names)
        dfs = []
        for idx, file_name in enumerate(file_names):
            try:
                with open(os.path.join(folder_path, file_name), 'r') as file:
                    lines = file.readlines()
                data = [[float(val) for val in line.split()] for line in lines]
                df = pd.DataFrame(data)
                if idx > 0:
                    df = df.drop(df.columns[0], axis=1)
                dfs.append(df)
            except Exception as e:
                print(f"Error reading {file_name}: {e}")

        final_df = pd.concat(dfs, axis=1)

        # Extract headers from the .inf file
        inf_file_path = os.path.join(folder_path, f'{folder}.inf')
        desc_values = ['time']
        with open(inf_file_path, 'r') as file:
            for line in file:
                desc = extract_desc(line)
                if desc:
                    desc_values.append(desc)

        num_cols = final_df.shape[1]
        num_headers = len(desc_values)

        if num_cols > num_headers:
            desc_values.extend(['unknown'] * (num_cols - num_headers))

        final_df.columns = desc_values

        # Save the NumPy array to a .npy file
        data_array = final_df.to_numpy()
        npy_file_path = os.path.join(data_directory, f'{folder}.npy')
        np.save(npy_file_path, data_array)
        print(f'Numpy .npy file saved for {folder}')


###Download NPY files

In [ ]:
import os
from google.colab import files
for filename in os.listdir(data_directory):
    if filename.endswith('.npy'):
        file_path = os.path.join(data_directory, filename)
        # Download the file
        files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Check NPY Format

In [ ]:
import pandas as pd
import numpy as np

# Step 1: Read CSV Data
df = pd.read_csv('/content/drive/MyDrive/dataCig/Bus23.csv')

# Step 2: Preprocess Data

# Step 3: Convert to NumPy Array
np_array = df.to_numpy()

# Step 4: Save as `.npy` File
np.save('Bus23.npy', np_array)


In [ ]:
import pandas as pd
import numpy as np

# Read the .csv file
csv_df = pd.read_csv('/content/drive/MyDrive/dataCig/Bus23.csv')

# Read the .npy file and convert it to a pandas DataFrame
npy_data = np.load('/content/Bus23.npy')
npy_df = pd.DataFrame(npy_data)

# Compare the two DataFrames
# Check if they have the same shape
same_shape = csv_df.shape == npy_df.shape

# Check if they have the same data values, ignoring column names
same_values = np.array_equal(csv_df.values, npy_df.values)

# Output the results
print("Same Shape:", same_shape)
print("Same Values:", same_values)


Same Shape: True
Same Values: True
